In [4]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import collections
import ast
import more_itertools as mit
from Bio.Seq import Seq
from Bio import SeqIO
import os
from Bio import pairwise2
from scipy.spatial import distance
from tqdm import tqdm

/home/mark/anaconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
speciesDict={}
input_file='/home/mark/Desktop/TE_Leaves/ls_primates/LS_MEI_Species_Sequences/'
for fasta in os.listdir(input_file):
    if 'Alu' in str(fasta):
        print(fasta)
        species = fasta.split("_")[0]
        speciesDict[species]=[]
        fasta_sequences = SeqIO.parse(open(input_file+fasta),'fasta')
        for fasta in fasta_sequences:
            name, sequence = fasta.id, str(fasta.seq)
            speciesDict[species].append(name)
    else:
        continue

GorGor_Alu.fasta
PanPan_Alu.fasta
PonPyg_Alu.fasta
SymSyn_Alu.fasta
PonAbe_Alu.fasta
hs1_Alu.fasta
PanTro_Alu.fasta


In [1]:
T2TDict={'NC_060925.1':'chr1',
'NC_060926.1':'chr2',
'NC_060927.1':'chr3',
'NC_060928.1':'chr4',
'NC_060929.1':'chr5',
'NC_060930.1':'chr6',
'NC_060931.1':'chr7',
'NC_060932.1':'chr8',
'NC_060933.1':'chr9',
'NC_060934.1':'chr10',
'NC_060935.1':'chr11',
'NC_060936.1':'chr12',
'NC_060937.1':'chr13',
'NC_060938.1':'chr14',
'NC_060939.1':'chr15',
'NC_060940.1':'chr16',
'NC_060941.1':'chr17',
'NC_060942.1':'chr18',
'NC_060943.1':'chr19',
'NC_060944.1':'chr20',
'NC_060945.1':'chr21',
'NC_060946.1':'chr22',
'NC_060947.1':'chrX',
'NC_060948.1':'chrY'}
T2TDict2 = {y:x for x,y in T2TDict.items()}

In [41]:
#directory='/home/mark/Desktop/TE_Leaves/ls_primates/PAV_SV_Runs/BorangRef/SecondFiltered/'
#outdirectory='/home/mark/Desktop/TE_Leaves/ls_primates/PAV_SV_Runs/BorangRef/TripleFiltered/'
directory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/'
outdirectory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part3_DuplicationFiltered/'

for csvFile in tqdm(os.listdir(directory)):
    
    print(directory)
    print(csvFile)
    
    if 'Filtered50Distance_06' in str(csvFile) and 'SymSyn' not in str(csvFile):
        
        species = str(csvFile.split("_")[0])
        df = pd.read_csv(directory+csvFile).set_index("ID")
        #break
        dupDF = df[df['UorD']=='Duplication'].copy()
        df['Duplication_Merged']='NONE'
        duplicateLoci={}
        flag=0
        
        
        if species !='GorGor' and species !='PanPan' and species !='hs1':
        
            for row in dupDF.index:
                secondFlag=0

                if flag==0:
                    duplicateLoci[row]=[]
                    flag+=1
                else:
                    contig = str(row.split(":")[0])
                    start = int(row.split(":")[1].split("-")[0])
                    end  = int(row.split(":")[1].split("-")[1])

                    for item in duplicateLoci.keys():
                        if secondFlag==0 and str(dupDF.at[item,str(species)+"-pri_Total_LowDistance_Hits"])!='NONE':
                            matches = ast.literal_eval(str(dupDF.at[item,str(species)+"-pri_Total_LowDistance_Hits"]))
                            for secondaryMatch in matches.keys():
                                secContig = str(secondaryMatch.split("_")[1].split(":")[0])
                                secStart=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[0])
                                secEnd=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[1])
                                
                                x = range(start,end)
                                y = range(secStart,secEnd)
                                xs = set(x)
                                if len(xs.intersection(y))>1 and secContig == contig:
                                    duplicateLoci[item].append(row)
                                    secondFlag=1
                                else:
                                    continue
                            

                        else:
                            continue
                            
                    if secondFlag!=0:
                        continue
                    else:
                        duplicateLoci[row]=[]
                        
                        
        elif  species =='hs1':
            for row in dupDF.index:
                    secondFlag=0

                    if flag==0:
                        duplicateLoci[row]=[]
                        flag+=1
                    else:
                        contig = str(row.split(":")[0])
                        start = int(row.split(":")[1].split("-")[0])
                        end  = int(row.split(":")[1].split("-")[1])

                        for item in duplicateLoci.keys():
                            if secondFlag==0 and str(dupDF.at[item,str(species)+"_Total_LowDistance_Hits"])!='NONE':
                                matches = ast.literal_eval(str(dupDF.at[item,str(species)+"_Total_LowDistance_Hits"]))
                                for secondaryMatch in matches.keys():
                                    secContig = T2TDict[str('_'.join(secondaryMatch.split("_")[1:]).split(":")[0])]
                                    secStart=int(secondaryMatch.split(":")[1].split("-")[0])
                                    secEnd=int(secondaryMatch.split(":")[1].split("-")[1])

                                    x = range(start,end)
                                    y = range(secStart,secEnd)
                                    xs = set(x)
                                    if len(xs.intersection(y))>1 and secContig == contig:
                                        duplicateLoci[item].append(row)
                                        secondFlag=1
                                    else:
                                        continue


                            else:
                                continue

                        if secondFlag!=0:
                            continue
                        else:
                            duplicateLoci[row]=[]
        
        
        
        else:
            
            for row in dupDF.index:
                secondFlag=0

                if flag==0:
                    duplicateLoci[row]=[]
                    flag+=1
                else:
                    contig = str(row.split(":")[0])
                    start = int(row.split(":")[1].split("-")[0])
                    end  = int(row.split(":")[1].split("-")[1])

                    for item in duplicateLoci.keys():
                        if secondFlag==0 and str(dupDF.at[item,str(species)+"-pat_Total_LowDistance_Hits"])!='NONE':
                            matches = ast.literal_eval(str(dupDF.at[item,str(species)+"-pat_Total_LowDistance_Hits"]))
                            for secondaryMatch in matches.keys():
                                secContig = str(secondaryMatch.split("_")[1].split(":")[0])
                                secStart=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[0])
                                secEnd=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[1])
                                
                                x = range(start,end)
                                y = range(secStart,secEnd)
                                xs = set(x)
                                if len(xs.intersection(y))>1 and secContig == contig:
                                    duplicateLoci[item].append(row)
                                    secondFlag=1
                                else:
                                    continue
                            

                        else:
                            continue
                            
                        if secondFlag==0 and str(dupDF.at[item,str(species)+"-mat_Total_LowDistance_Hits"])!='NONE':
                            matches = ast.literal_eval(str(dupDF.at[item,str(species)+"-mat_Total_LowDistance_Hits"]))
                            for secondaryMatch in matches.keys():
                                secContig = str(secondaryMatch.split("_")[1].split(":")[0])
                                secStart=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[0])
                                secEnd=int(secondaryMatch.split("_")[1].split(":")[1].split("-")[1])
                                
                                x = range(start,end)
                                y = range(secStart,secEnd)
                                xs = set(x)
                                if len(xs.intersection(y))>1 and secContig == contig:
                                    duplicateLoci[item].append(row)
                                    secondFlag=1
                                else:
                                    continue
                            

                        else:
                            continue
                            
                    if secondFlag!=0:
                        continue
                    else:
                        duplicateLoci[row]=[]

                    
                    
                    
        
        
        
        
        dropList=[]
        for key in duplicateLoci.keys():
            if len(duplicateLoci[key])>0:
                df.at[key,'Duplication_Merged']=duplicateLoci[key]
            else:
                continue
            
        goodRows=[]
        for row in df.index:
            if str(df.at[row,'Duplication_Merged']) == 'NONE' and str(df.at[row,'UorD']) == 'Duplication':
                continue
            else:
                goodRows.append(row)
        cleanedMergedDF = df.loc[goodRows].copy()
        cleanedMergedDF.to_csv(outdirectory+str(species)+"_MergedDup_Clean_Final_06-26-2024.csv")

  0%|                                                     | 0/7 [00:00<?, ?it/s]

/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
SymSyn_Filtered50Distance_06_25_2024.csv
/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
PonAbe_Filtered50Distance_06_25_2024.csv


 29%|████████████▊                                | 2/7 [00:00<00:00,  5.02it/s]

/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
GorGor_Filtered50Distance_06_25_2024.csv


 43%|███████████████████▎                         | 3/7 [00:01<00:01,  2.65it/s]

...

 71%|████████████████████████████████▏            | 5/7 [00:01<00:00,  2.99it/s]

/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
PanPan_Filtered50Distance_06_25_2024.csv


 86%|██████████████████████████████████████▌      | 6/7 [00:02<00:00,  1.84it/s]

/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
hs1_Filtered50Distance_06_25_2024.csv


100%|█████████████████████████████████████████████| 7/7 [00:03<00:00,  1.83it/s]


In [13]:
#directory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part3_DuplicationFiltered/'
#outdirectory='/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Part4_FilteredandFixed/'

for csvFile in os.listdir(directory):
    
    print(directory)
    print(csvFile)
    
    if 'Filtered50Distance_06' in str(csvFile) and 'SymSyn' not in str(csvFile):
        
        species = str(csvFile.split("_")[0])
        df = pd.read_csv(directory+csvFile).set_index("ID")
        #break
        dupDF = df[df['All_Matches']=='TRUE_INSERTION'].copy()
        df['Duplication_Hamming_Merged']='NONE'
        duplicateLoci={}
        flag=0

        for row in tqdm(dupDF.index):
            #print(row)

            if flag==0:

                duplicateLoci[row]={'Merge':{}, 'Sequence':dupDF.at[row,'Sequence'], 'Element':str(dupDF.at[row,'TE_Designation'])}
                flag=1

            else:

                ## If flag ==1
                mergedFlag=0
                for loci in duplicateLoci.keys():
                    #print(loci)
                    #print(duplicateLoci[loci])

                    if mergedFlag==0 and str(dupDF.at[row,'TE_Designation']) == str(duplicateLoci[loci]['Element']):

                        lociSequence = str(duplicateLoci[loci]['Sequence'])

                        senseSequence = str(dupDF.at[row,'Sequence'])
                        antiSequence =  str(Seq(dupDF.at[row,'Sequence']).reverse_complement())

                        #Sense Alignment
                        alignments = pairwise2.align.globalxs(senseSequence, lociSequence, -1, -1)
                        distances=[]
                        for alignment in alignments[0]: 
                            seq1=[x for x in alignment.seqA]
                            seq2 =[y for y in alignment.seqB]
                            distances.append(distance.hamming(seq1, seq2))
                            break

                        sensebestMatch = min(distances)


                        #Antisense Alignment
                        aalignments = pairwise2.align.globalxs(antiSequence, lociSequence, -1, -1)
                        adistances=[]
                        for alignment in aalignments[0]: 
                            seq1=[x for x in alignment.seqA]
                            seq2 =[y for y in alignment.seqB]
                            adistances.append(distance.hamming(seq1, seq2))
                            break

                        antisensebestMatch = min(adistances)

                        if float(sensebestMatch)<=0.05 or float(antisensebestMatch)<=0.05:

                            mergedFlag=1
                            if float(antisensebestMatch)<float(sensebestMatch):
                                bestoverallMatch =float(antisensebestMatch)
                            else:
                                bestoverallMatch =float(sensebestMatch)

                            duplicateLoci[loci]['Merge'][row]=bestoverallMatch

                        else:

                            continue

                    else:
                        continue

                if mergedFlag==0:
                    duplicateLoci[row]={'Merge':{}, 'Sequence':dupDF.at[row,'Sequence'], 'Element':str(dupDF.at[row,'TE_Designation'])}
                else:
                    continue
        dropList=[]
        for key in duplicateLoci.keys():
            df.at[key,'Duplication_Hamming_Merged']=duplicateLoci[key]['Merge']
            for locus in duplicateLoci[key]['Merge'].keys():
                dropList.append(locus)
        cleanedMergedDF = df.drop(dropList).copy()
        cleanedMergedDF.to_csv(outdirectory+str(species)+"_MergedDup_Clean_Final_06-26-2024.csv")

/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
SymSyn_Filtered50Distance_06_25_2024.csv
/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
PonAbe_Filtered50Distance_06_25_2024.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43/43 [06:36<00:00,  9.21s/it]


/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
GorGor_Filtered50Distance_06_25_2024.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:41<00:00,  1.37s/it]


/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
PonPyg_Filtered50Distance_06_25_2024.csv


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:49<00:00,  1.31s/it]


/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
PanPan_Filtered50Distance_06_25_2024.csv


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 70/70 [05:29<00:00,  4.71s/it]


/home/mark/Desktop/TE_Leaves/ls_primates/LS_Dataframes/Filtered50Distance/
hs1_Filtered50Distance_06_25_2024.csv


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 192/192 [14:11<00:00,  4.44s/it]


In [45]:
senseSequence='AAAT'
lociSequence='AAT'

In [47]:
alignments = pairwise2.align.globalxs(senseSequence, lociSequence, -1, -1)
distances=[]
for alignment in alignments: 
    seq1=[x for x in alignment.seqA]
    seq2 =[y for y in alignment.seqB]
    distances.append(distance.hamming(seq1, seq2))
    break

In [48]:
alignment

Alignment(seqA='AAAT', seqB='-AAT', score=2.0, start=0, end=4)

In [49]:
seq1

['A', 'A', 'A', 'T']

In [50]:
seq2

['-', 'A', 'A', 'T']